## Multiclass Logistic Regression

In [12]:
import numpy as np

class softmax_regression():
    
    def __init__(self):
        
        self.W = None
        
        
    @staticmethod
    def softmax(arr):
        """
        Rowwise computation of softmax.
        Input:
         - arr: an array of dimension n x k (in this context X @ W.T)
        Output:
         - an array of the same dimension as input
        """
        a = arr.max(axis=1)
        ex_arr = np.exp(arr - a[:,None])
        return ex_arr / ex_arr.sum(axis=1)[:,None]
    
    @staticmethod
    def log_sum_exp(arr):
        """
        Rowwise computation of log-sum-exp.
        Input: 
         - arr: an array of dimension n x k (in this context X @ W.T)
        Output:
         - an array of dimension n x 1
        """
        a = arr.max(axis=1)
        return a + np.log(np.sum(np.exp(arr - a[:,None]), axis=1))
    
    @staticmethod
    def one_hot(vec):
        """
        One hot encoder for response:
        Input:
         - vec: an n x 1 (response) vector 
        Output:
         - an n x k one hot encoding of the input
        """
        # one hot encoding from: 
        #  - https://stackoverflow.com/questions/29831489/numpy-1-hot-array
        one_hot = np.zeros((len(vec), len(np.unique(vec))))
        one_hot[np.arange(len(vec)), vec] = 1
        return one_hot
    
    @staticmethod
    def gradient_descent(f, grad_f, w0, alpha=0.005, eps=0.01):
        """
        Gradient Descent for fitting:
        Input: 
         - f:      the function to be minimized (loss in this context)
         - grad_f: the gradient of the function to be minimized
         - w0:     an initial guess
         - alpha:  the learning rate (constant at this point)
         - eps:    the tolerance on the magnitude of the gradient
        Outputs:
         - a k x d dimensional array of weights w that minimizes f
        """
        w = w0
        g = grad_f(w)
        while np.linalg.norm(g) > eps:  
            g = grad_f(w)
            w = w - alpha * grad_f(w)
            #print(np.linalg.norm(g))
        return w
    
    @staticmethod
    def loss(W, X, y):
        """
        Loss for Logistic Regression 
        Inputs:
         - W: a k x d weight matrix where rows are the d weights for kth class
         - X: the n x d data matrix (n observations, d features)
         - y: the target classes
        Outputs:
         - a number that should mean something to someone
        """
        one_hot = softmax_regression.one_hot(y)

        W = W.reshape((len(np.unique(y)), X.shape[1]))
        xw = X @ W.T 
        lse = softmax_regression.log_sum_exp(xw)
        return -np.sum(one_hot * (xw - lse[:,None])) 
    
    @staticmethod
    def grad_loss(W, X, y):
        """
        Inputs:
         - W: k x d array of parameters
         - X: n x d design matrix
         - y: n x 1 target vector
        Output:
         - a k x d array of partial derivatives
        """
        one_hot = softmax_regression.one_hot(y)

        W = W.reshape((len(np.unique(y)), X.shape[1]))

        g = X.T @ (one_hot - softmax_regression.softmax(X @ W.T)) 
        return -g.T #-g.flatten(order='F')

    def fit(self, X, y):
        """
        Fit method:
        Input:
         - X: an n x d design matrix
         - y: a n x 1 target vector
        Ouput:
         - a k x d array of optimal weights
        """
        f = lambda w: softmax_regression.loss(w, X, y)
        grad_f = lambda w: softmax_regression.grad_loss(w, X, y)
        w0 = np.random.rand(len(np.unique(y))*X.shape[1])
        w0 = w0.reshape((len(np.unique(y)), X.shape[1]))
        self.W = softmax_regression.gradient_descent(f, grad_f, w0)
        return None
        
        
    def predict(self, X):
        """
        Multiclass soft prediction method:
        Input:
         - X: an n x d design matrix
        Output:
         - a n x k array of probabilistic predictions
           each observation gets k predicted probabilities 
         
         - note: if you want hard predictions, use out.argmax(axis=1)
           where out is what this method returns
        """
        return softmax_regression.softmax(X @ self.W.T)
    

In [22]:
# for comparison
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import sklearn.datasets
import matplotlib.pyplot as plt

In [23]:
# load some dataset to check how it works
digits = sklearn.datasets.load_digits()
X, Xvalidate, y, yvalidate = train_test_split(digits.data, digits.target)

In [48]:
# fit my implementation and sklearn's version
my_clf = softmax_regression()
clf = LogisticRegression(multi_class='multinomial', solver='saga', C=1000)
my_clf.fit(X,y)
clf.fit(X,y)

/home/bradley/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

In [49]:
# my implementations accuracy:
train_pred = my_clf.predict(X).argmax(axis=1)
val_pred = my_clf.predict(Xvalidate).argmax(axis=1)

my_train = np.sum(train_pred == y) / len(y)
my_val = np.sum(val_pred == yvalidate) / len(yvalidate)

# sklearn's accuracy:
sk_train = clf.score(X, y)
sk_val = clf.score(Xvalidate, yvalidate)

In [50]:
print("My implementation training accuracy: {}".format(100*my_train))
print("My implementation validation accuracy: {}\n".format(100*my_val))
print("Sklearn's training accuracy: {}".format(100*sk_train))
print("Sklearn's validation accuracy: {}".format(100*sk_val))

My implementation training accuracy: 100.0
My implementation validation accuracy: 96.22222222222221

Sklearn's training accuracy: 100.0
Sklearn's validation accuracy: 97.33333333333334


So my implementation is not doing as well as sklearn's (which is no surprise). At this stage I do not have an explaination for this but hopefully I can find a reason in sklearn's source code.

To do:

- Stochastic gradient descent.
- Line search for step size descent algorithm(s).
- Change of basis functionality for nonlinear boundaries.